# Augmentation 전략
1. Question이 부정문 혹은 반대 의미로 쓸 수 있는 문장을 찾는다
2. 부정문을 긍정문으로 바꾸고, 반대의미의 문장을 생성한다.
3. a, b, c, d 중 answer을 제외한 선지를 2.의 정답으로 하는 데이터 생성

# 데이터 로드

In [ ]:
# 구글드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ssafy_ai_chllenge/train.csv")
test_df  = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ssafy_ai_chllenge/test.csv")

In [ ]:
train_df.head(20)

,id,path,question,a,b,c,d,answer
0,train_0001,train/train_0001.jpg,이 사진 속 운동기구가 설치된 장소는 어디일까요?,학교 운동장,공원,헬스장 내부,쇼핑몰 내부,b
1,train_0002,train/train_0002.jpg,이 사진에 보이는 전통 한국 건축물은 무엇인가요?,궁궐,성,사찰,한옥,d
2,train_0003,train/train_0003.jpg,이 사진에서 보이는 탈것은 무엇인가요?,세발 오토바이,오토바이,자동차,자전거,a
3,train_0004,train/train_0004.jpg,이 사진에서 보이는 새는 무엇인가요?,참새,갈매기,백로,오리,c
4,train_0005,train/train_0005.jpg,이 사진에서 사람들이 모여서 보고 있는 것은 무엇인가요?,해수욕,바다 축제,불꽃놀이,해돋이,d
5,train_0006,train/train_0006.jpg,이 사진에서 보이는 강 위를 건너기 위해 설치된 시설은 무엇인가요?,징검다리,배,다리,터널,a
6,train_0007,train/train_0007.jpg,이 사진에서 볼 수 있는 것은 무엇인가요?,강가의 배,도시의 고층 건물,꽃이 만발한 나무와 돌담,사막의 모래언덕,c
7,train_0008,train/train_0008.jpg,이 사진 속에서 볼 수 없는 것은 무엇인가요?,스탠드 조명,컴퓨터,의자,책,b
8,train_0009,train/train_0009.jpg,이 도로에서 제한 속도는 얼마인가요?,50km/h,60km/h,30km/h,40km/h,d
9,train_0010,train/train_0010.jpg,이 공간의 벽에 주로 무엇이 장식되어 있나요?,식물과 꽃,그림과 조각품,거울과 액자,사진과 포스터,d


In [ ]:
train_df.iloc[150:200]['question']

In [ ]:
opp_word = ['않는', '지 않은', '아닌', '없는','높은가','적절한','된 색상','보이는']

# Create a boolean mask to filter rows
mask = train_df['question'].str.contains('|'.join(opp_word))

# Create a new DataFrame with the filtered rows
neg_df = train_df[mask].copy()

# Display the new DataFrame
display(neg_df.head())

# Filtering and Save

In [ ]:
from tqdm import tqdm
aug_df = pd.DataFrame(columns = ['id',  'path', 'question', 'a', 'b', 'c', 'd', 'answer'])
aug_rows =[]
opp_word_dict = {
    '지 않는': '는',
    '지 않은' : '는',
    '아닌': '맞는',
    '없는': '있는',
    '높은가': '낮은가',
    '적절한': '적절하지 않은',
    '된 색상': '되지 않은 색상',
    '보이는 ': '',
}
# 3. 데이터프레임 순회 및 새로운 행 생성
for index, row in tqdm(neg_df.iterrows()):
    origin_answer = set([row['answer']])
    origin_question = row['question']
    new_questions = []
    for word in opp_word_dict.keys():
        if word in origin_question:
            new_question = origin_question[:]
            if '보이는 ' == word:
                new_questions.append((1, new_question.replace(word, opp_word_dict[word])))
            else:
                new_questions.append((0, new_question.replace(word, opp_word_dict[word])))

    options = set(['a', 'b', 'c', 'd'])
    other_options = options - origin_answer
    for see, new_q in new_questions:
        if see == 1:
            new_row = row.copy()
            new_row['question'] = list(origin_answer)[0]
            continue
        for option in other_options:
            new_row = row.copy()
            new_row['question'] = new_q
            new_row['answer'] = option
            aug_rows.append(new_row.tolist())


In [ ]:
aug_df = pd.DataFrame(aug_rows, columns = ['id',  'path', 'question', 'a', 'b', 'c', 'd', 'answer'])
print(aug_df.head())

In [ ]:
len(train_df)

In [ ]:
len(aug_df)

In [ ]:
aug_df_final = pd.concat([train_df,aug_df])
print(len(aug_df_final))

In [ ]:
aug_df_final.to_csv('/content/drive/MyDrive/Colab Notebooks/ssafy_ai_chllenge/train_aug.csv')

# Task type hint 저장

In [ ]:
train_df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ssafy_ai_chllenge/train.csv")
test_df  = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ssafy_ai_chllenge/test.csv")
train_df_hint = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ssafy_ai_chllenge/train_hint.csv")
test_df_hint  = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ssafy_ai_chllenge/test_hint.csv")

In [ ]:
# Rule-based task_type classifier for the user's CSV at /mnt/data/task.csv
import re
import pandas as pd
from pathlib import Path

# ===== 1) Keyword dictionaries =====
COLOR_WORDS = {
    "ko": ["색", "색깔", "무슨 색", "어떤 색", "빛깔", "컬러"],
    "en": ["color", "colour", "what color"]
}
COUNT_WORDS = {
    "ko": ["몇 개", "개수", "얼마나", "몇명", "몇 명", "몇마리", "몇 마리", "수량"],
    "en": ["how many", "count", "number of"]
}
TEXT_OCR_WORDS = {
    "ko": ["글자", "문구", "표지판", "간판", "텍스트", "문자", "읽", "라벨", "번호판", "표기"],
    "en": ["text", "word", "words", "label", "read", "sign", "plate", "license"]
}
CLOCK_TIME_WORDS = {
    "ko": ["시계", "시각", "몇 시", "몇시", "시간", "시침", "분침"],
    "en": ["clock", "time", "o'clock", "what time"]
}
POSITION_WORDS = {
    "ko": ["왼쪽", "오른쪽", "가운데", "중앙", "상단", "하단", "옆", "뒤", "앞"],
    "en": ["left", "right", "center", "middle", "top", "bottom", "behind", "front", "near"]
}
SCENE_LOC_WORDS = {
    "ko": ["어디", "어디인가요", "장소", "장면", "도시", "국가", "지역", "현장", "배경"],
    "en": ["where", "place", "scene", "location", "city", "country"]
}
OBJECT_WORDS = {
    "ko": ["무엇", "무엇인가요", "무슨 물체", "어떤 물체", "종류", "타입"],
    "en": ["what is this", "what object", "which object", "what animal", "type of"]
}
ACTION_WORDS = {
    "ko": ["무엇을 하고", "하고 있", "동작", "행동", "중", "하는 중"],
    "en": ["doing", "action", "is he", "is she", "are they", "doing what"]
}
ATTRIBUTE_WORDS = {
    "ko": ["크기", "길이", "높이", "넓이", "패턴", "모양", "재질", "형태"],
    "en": ["size", "length", "height", "width", "pattern", "shape", "material", "texture"]
}
WEATHER_WORDS = {
    "ko": ["날씨", "비", "눈", "맑", "흐림", "구름"],
    "en": ["weather", "rain", "snow", "sunny", "cloudy"]
}

COLOR_CHOICES = [
    "red","blue","green","yellow","black","white","gray","grey","brown","pink","purple","orange",
    "빨강","파랑","초록","노랑","검정","흰색","회색","갈색","분홍","보라","주황"
]
DIRECTION_CHOICES = ["left","right","center","middle","top","bottom","왼쪽","오른쪽","가운데","중앙","상단","하단"]
BOOLEAN_CHOICES = ["yes","no","참","거짓","맞다","아니다","있다","없다"]
CITY_COUNTRY_HINT = ["seoul","tokyo","london","paris","new york","베이징","서울","도쿄","런던","파리","뉴욕","중국","한국","일본","프랑스","미국"]

def ratio_numeric(tokens):
    c = 0
    for t in tokens:
        t = str(t).strip()
        if re.fullmatch(r"[0-9]+(\.[0-9]+)?", t):
            c += 1
    return c / max(1, len(tokens))

def contains_any(text, words):
    text = "" if text is None else str(text)
    text_l = text.lower()
    return any(w in text_l for w in words["en"]) or any(w in text for w in words["ko"])

def list_contains_any(options, vocab):
    opts = [str(x).strip().lower() for x in options]
    return any(any(v in o for v in vocab) for o in opts)

def majority_in_vocab(options, vocab):
    opts = [str(x).strip().lower() for x in options]
    if not opts: return False
    cnt = sum(1 for o in opts if any(v == o or v in o for v in vocab))
    return cnt >= (len(opts) // 2 + 1)

TASK_HINT = {
    "color": "주요 대상의 표면/의류/신호등 등 색상이 뚜렷한 영역을 우선 주시하고, 그림자/광원 영향이 적은 부분의 색을 비교하세요.",
    "count": "화면 전역을 훑으며 동일 객체 인스턴스를 식별하고 겹침/가림 영역을 주의 깊게 스캔하세요.",
    "text_ocr": "고대비 문양/평평한 표면(간판, 라벨, 표지판, 차량 번호판)을 클로즈업하듯 주시해 글자 윤곽을 확인하세요.",
    "clock_time": "아날로그 시계의 시침/분침 끝을 집중 관찰하거나 디지털의 숫자 세그먼트를 확인하세요.",
    "position": "질문에 언급된 방향(왼쪽/오른쪽/상단/하단/가운데) 영역을 먼저 살피세요.",
    "scene_location": "이미지 전체 레이아웃과 배경 단서를 우선 관찰하세요(건축물 유형, 도로 표지, 자연물 등).",
    "object_classification": "프레임의 중심 또는 가장 salient한 물체 외곽선/형태를 우선 관찰하세요.",
    "action": "사람/동물의 포즈, 팔/손/시선, 상호작용 물체(도구, 공 등) 주변을 집중 관찰하세요.",
    "attribute": "대상의 가장 특징적인 부분(무늬/재질/형태 대비가 큰 부분)을 확대하듯 살피세요.",
    "weather": "하늘/지평선/지면 상태를 확인하고 강수/적설/구름 분포 단서를 관찰하세요.",
    "boolean": "질문의 대상 영역을 먼저 찾고, 존재/부재나 상태를 빠르게 확인하세요.",
    "unknown": "이미지 전역을 스캔한 뒤 질문 키워드와 매칭되는 후보 영역을 단계적으로 좁혀보세요."
}

PRIORITY = [
    "color", "text_ocr", "clock_time", "count", "position",
    "scene_location", "action", "attribute", "object_classification",
    "weather", "boolean"
]

def infer_task_type(question, choices):
    q = (question or "").strip()
    a,b,c,d = choices
    options = [a,b,c,d]

    # 1) options-driven
    if majority_in_vocab(options, COLOR_CHOICES):
        return "color", "선지에 색상 단어 다수"
    if majority_in_vocab(options, DIRECTION_CHOICES):
        return "position", "선지에 방향/위치 단어 다수"
    if ratio_numeric(options) >= 0.5:
        if contains_any(q, CLOCK_TIME_WORDS):
            return "clock_time", "선지 숫자 + 질문에 시계/시간"
        return "count", "선지에 숫자 다수"
    if majority_in_vocab(options, BOOLEAN_CHOICES):
        if contains_any(q, TEXT_OCR_WORDS):
            return "text_ocr", "Yes/No + 텍스트 판독 질문"
        if contains_any(q, COUNT_WORDS):
            return "count", "Yes/No지만 개수 관련"
        return "boolean", "선지가 예/아니오 계열"
    if list_contains_any(options, CITY_COUNTRY_HINT):
        return "scene_location", "선지에 도시/국가명"

    # 2) question-driven
    if contains_any(q, COLOR_WORDS): return "color", "질문에 색 관련 키워드"
    if contains_any(q, TEXT_OCR_WORDS): return "text_ocr", "질문에 텍스트/표지/읽기"
    if contains_any(q, CLOCK_TIME_WORDS): return "clock_time", "질문에 시계/시간"
    if contains_any(q, COUNT_WORDS): return "count", "질문에 개수/몇 개"
    if contains_any(q, POSITION_WORDS): return "position", "질문에 위치/방향"
    if contains_any(q, SCENE_LOC_WORDS): return "scene_location", "질문에 장소/어디"
    if contains_any(q, ACTION_WORDS): return "action", "질문에 동작/무엇을 하고"
    if contains_any(q, ATTRIBUTE_WORDS): return "attribute", "질문에 속성(크기/모양/재질)"
    if contains_any(q, OBJECT_WORDS): return "object_classification", "질문에 무엇/어떤 물체"
    if contains_any(q, WEATHER_WORDS): return "weather", "질문에 날씨"

    # backup
    if list_contains_any(options, COLOR_CHOICES):
        return "color", "선지에 색상 단어 포함"
    if list_contains_any(options, DIRECTION_CHOICES):
        return "position", "선지에 방향 단어 포함"

    return "unknown", "규칙에 매칭되지 않음"

def task_hint(task_type):
    return TASK_HINT.get(task_type, TASK_HINT["unknown"])


In [ ]:
def detect_q_and_choices(df: pd.DataFrame):
    """question, a,b,c,d 컬럼을 휴리스틱으로 감지"""
    cols = list(df.columns)
    lower_map = {c.lower(): c for c in cols}

    # question 후보
    candidate_q_cols = [c for c in cols
                        if c.lower() in ["question", "질문"]
                        or "question" in c.lower()
                        or "질문" in c.lower()]
    q_col = candidate_q_cols[0] if candidate_q_cols else ("question" if "question" in lower_map else cols[0])

    # 선택지 후보
    choice_candidates = []
    for base in ["a", "b", "c", "d"]:
        if base in lower_map:
            choice_candidates.append(lower_map[base])
        else:
            hit = [c for c in cols if c.lower()==base
                   or c.lower().endswith("_"+base)
                   or c.lower().startswith(base+"_")
                   or c.lower().strip()==base]
            if hit:
                choice_candidates.append(hit[0])
            else:
                hit = [c for c in cols if base in c.lower() and ("choice" in c.lower() or "보기" in c.lower())]
                if hit:
                    choice_candidates.append(hit[0])

    if len(choice_candidates) < 4:
        remain = [c for c in cols if c != q_col]
        choice_candidates = remain[:4]

    if len(choice_candidates) != 4:
        raise ValueError(f"선지 열 4개(a,b,c,d)를 찾지 못했습니다. 감지된 후보: {choice_candidates} | 전체 컬럼: {cols}")

    a_col, b_col, c_col, d_col = choice_candidates
    return q_col, a_col, b_col, c_col, d_col


def classify_df(df: pd.DataFrame):
    """df에 task_type / task_reason / task_hint 컬럼을 추가해서 반환 + 간단 리포트"""
    q_col, a_col, b_col, c_col, d_col = detect_q_and_choices(df)

    task_types, reasons, hints = [], [], []
    for _, row in df.iterrows():
        q = str(row[q_col])
        a, b, c, d = (row[a_col], row[b_col], row[c_col], row[d_col])
        t, why = infer_task_type(q, (a, b, c, d))
        task_types.append(t)
        reasons.append(why)
        hints.append(task_hint(t))

    out = df.copy()
    out["task_type"] = task_types
    out["task_reason"] = reasons
    out["task_hint"] = hints

    counts = out["task_type"].value_counts(dropna=False)
    meta = {
        "q_col": q_col,
        "choice_cols": [a_col, b_col, c_col, d_col],
        "counts": counts
    }
    return out, meta

train_df_tasktype, meta_train = classify_df(train_df)

test_df_tasktype, meta_test = classify_df(test_df)

In [ ]:
train_df_hint['task_hint'] = train_df_tasktype['task_hint']
test_df_hint['task_hint'] = test_df_tasktype['task_hint']

In [ ]:
train_df_hint.to_csv("/content/drive/MyDrive/Colab Notebooks/ssafy_ai_chllenge/train_hint_tasktype.csv")
test_df_hint.to_csv("/content/drive/MyDrive/Colab Notebooks/ssafy_ai_chllenge/test_hint_tasktype.csv")